In [1]:
def strategy(df, trigger, window=1):
    df = df.copy()
    df['log_ret'] = df.Close.apply(np.log).diff()
    df['prior_n'] = df.log_ret.rolling(window).sum()
    df.dropna(inplace = True)

    df['position'] = [1 if i>trigger else 0 for i in df.prior_n] 
    ## Long, short or neutral
    
    df['log_strat'] = df.position.shift(1) * df.log_ret

    # number of trades
    df['trades'] = 0
    trades = 0
    for i in range(len(df['position']) - 1):
        if df['position'].iloc[i] != df['position'].iloc[i+1] : 
            trades += 1
        df.iloc[i+1, df.columns.get_loc('trades')] = trades

    return df

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
df = yf.download('BTC-USD', start = '2022-12-3', interval = '1d',auto_adjust=True)
df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2022-12-03,16908.236328,17116.041016,16888.140625,17090.097656,16217776704
2022-12-04,17130.486328,17157.767578,16903.439453,16908.169922,16824520830
2022-12-05,16974.826172,17378.152344,16922.431641,17128.894531,22209086834
2022-12-06,17089.503906,17091.863281,16939.921875,16975.238281,19889922369
2022-12-07,16848.126953,17109.376953,16750.558594,17089.505859,19675404389
...,...,...,...,...,...
2025-12-13,90298.710938,90647.570312,89800.992188,90257.796875,64237748110
2025-12-14,88175.179688,90498.109375,87634.937500,90296.437500,50465972205


In [3]:
results = []#pd.DataFrame(columns = ['trigger', 'window', 'return', 'volatility' , 'trades' , 'asset return' , 'asset vol'])

for trigger in np.linspace(0.001, 0.01, 5):
    for window in np.linspace( 3 , 11, 6, dtype=int):
        df_i = strategy(df, trigger, window)
      
        vol = df_i[['log_strat']].std()*len(df)**.5 
        avol = df_i[['log_ret']].std()*len(df)**.5 
        strat_ret = np.exp( df_i['log_strat'].sum() ) - 1 
        asset_ret = np.exp( df_i['log_ret'].sum() ) - 1 
    
        #results.loc[len(results)] = [round(trigger, 4), int(window), float(strat_ret), float(vol.iloc[0]), df_i.trades.iloc[-1],                                     float(asset_ret), float(avol.iloc[0])]
        results.append({
            'trigger': round(trigger, 4),
            'window': int(window),
            'return': float(strat_ret),
            'volatility': float(vol.iloc[0]),
            'trades': df_i.trades.iloc[-1],
            'asset return': float(asset_ret),
            'asset vol': float(avol.iloc[0])
        })

    # print('Trigger ', round(trigger,4))

results = pd.DataFrame(results)

print(results)


    trigger  window    return  volatility  trades  asset return  asset vol
0    0.0010       3  2.718182    0.594506     339      4.103988   0.807750
1    0.0010       4  2.702846    0.570550     276      4.069738   0.808098
2    0.0010       6  1.137626    0.574387     221      4.027385   0.808398
3    0.0010       7  0.817582    0.574310     197      4.056822   0.808731
4    0.0010       9  1.307928    0.584622     165      4.065384   0.809458
5    0.0010      11  0.873035    0.580164     169      3.872490   0.809566
6    0.0032       3  2.489852    0.574387     321      4.103988   0.807750
7    0.0032       4  2.507361    0.565227     288      4.069738   0.808098
8    0.0032       6  0.910178    0.566142     229      4.027385   0.808398
9    0.0032       7  0.945606    0.557569     201      4.056822   0.808731
10   0.0032       9  1.171597    0.580870     161      4.065384   0.809458
11   0.0032      11  0.849556    0.577352     175      3.872490   0.809566
12   0.0055       3  1.67